In [1]:
import requests
import pandas as pd

# Create a session
session = requests.Session()

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.nseindia.com/option-chain",
    "Host": "www.nseindia.com"
}

# Step 1: Visit NSE's option chain page to get cookies
session.get("https://www.nseindia.com/option-chain", headers=headers)

# Step 2: Fetch the option chain data
url = "https://www.nseindia.com/api/option-chain-equities?symbol=RELIANCE"
response = session.get(url, headers=headers)

# Check if the request was successful
if response.status_code != 200:
    raise Exception(f"Failed to fetch data: HTTP {response.status_code}")

# Step 3: Load JSON safely
try:
    data = response.json()
except ValueError:
    print("Response content is not JSON. Here's what came back:")
    print(response.text)
    raise

# Step 4: Parse into DataFrame
records = []
for rec in data.get("records", {}).get("data", []):
    strike = rec.get("strikePrice")
    expiry = rec.get("expiryDate")
    ce = rec.get("CE", {})
    pe = rec.get("PE", {})
    records.append({
        "Strike": strike,
        "Expiry": expiry,
        "Call OI": ce.get("openInterest"),
        "Call LTP": ce.get("lastPrice"),
        "Put OI": pe.get("openInterest"),
        "Put LTP": pe.get("lastPrice")
    })

df = pd.DataFrame(records)
print(df)

# Save to CSV
df.to_csv("reliance_option_chain.csv", index=False)


    Strike       Expiry  Call OI  Call LTP  Put OI  Put LTP
0     1140  28-Oct-2025      7.0    268.00   132.0     0.10
1     1160  28-Oct-2025      9.0    228.60    52.0     0.20
2     1160  25-Nov-2025      0.0      0.00     0.0     0.00
3     1160  30-Dec-2025      NaN       NaN     0.0     0.00
4     1180  28-Oct-2025   1163.0    222.00    91.0     0.20
..     ...          ...      ...       ...     ...      ...
93    1600  28-Oct-2025   2381.0      0.50  2012.0   181.50
94    1600  25-Nov-2025    653.0      2.45    71.0   173.85
95    1600  30-Dec-2025      1.0      6.45     NaN      NaN
96    1620  28-Oct-2025    114.0      0.35    71.0   200.00
97    1640  28-Oct-2025    315.0      0.15  1809.0   219.70

[98 rows x 6 columns]


In [3]:
import requests
import pandas as pd
import time

# Create a session
session = requests.Session()

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.nseindia.com/option-chain",
    "Host": "www.nseindia.com"
}

# Step 1: Visit NSE's option chain page to get cookies
session.get("https://www.nseindia.com/option-chain", headers=headers)

# Optional small delay to ensure cookies are set
time.sleep(1)

# Step 2: Fetch the option chain data
url = "https://www.nseindia.com/api/option-chain-equities?symbol=RELIANCE"
response = session.get(url, headers=headers)
data = response.json()

# Step 3: Loop over each expiry date and save separate files
expiry_dates = data.get("records", {}).get("expiryDates", [])

for expiry in expiry_dates:
    records = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {})
            pe = rec.get("PE", {})
            records.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call OI": ce.get("openInterest"),
                "Call LTP": ce.get("lastPrice"),
                "Put OI": pe.get("openInterest"),
                "Put LTP": pe.get("lastPrice")
            })

    df = pd.DataFrame(records)
    file_name = f"reliance_option_chain_{expiry.replace('-', '')}.csv"
    df.to_csv(file_name, index=False)
    print(f"Saved {file_name} with {len(df)} rows.")

Saved reliance_option_chain_28Oct2025.csv with 42 rows.
Saved reliance_option_chain_25Nov2025.csv with 37 rows.
Saved reliance_option_chain_30Dec2025.csv with 19 rows.


In [5]:
import requests
import pandas as pd
import time

# Create a session
session = requests.Session()

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.nseindia.com/option-chain",
    "Host": "www.nseindia.com"
}

# Step 1: Visit NSE's option chain page to get cookies
session.get("https://www.nseindia.com/option-chain", headers=headers)

# Small delay to ensure cookies are set
time.sleep(1)

# Step 2: Fetch the option chain data
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
response = session.get(url, headers=headers)
data = response.json()

# Step 3: Loop over each expiry date and save separate files + calculate PCR
expiry_dates = data.get("records", {}).get("expiryDates", [])

pcr_results = []  # store PCR per expiry

for expiry in expiry_dates:
    records = []
    total_call_oi = 0
    total_put_oi = 0

    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {})
            pe = rec.get("PE", {})

            call_oi = ce.get("openInterest", 0)
            put_oi = pe.get("openInterest", 0)

            total_call_oi += call_oi if call_oi else 0
            total_put_oi += put_oi if put_oi else 0

            records.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call OI": call_oi,
                "Call LTP": ce.get("lastPrice"),
                "Put OI": put_oi,
                "Put LTP": pe.get("lastPrice")
            })

    # Save CSV for this expiry
    df = pd.DataFrame(records)
    file_name = f"reliance_option_chain_{expiry.replace('-', '')}.csv"
    df.to_csv(file_name, index=False)
    print(f"Saved {file_name} with {len(df)} rows.")

    # Calculate PCR
    pcr = round(total_put_oi / total_call_oi, 3) if total_call_oi else None
    pcr_results.append({"Expiry": expiry, "Total Call OI": total_call_oi, "Total Put OI": total_put_oi, "PCR": pcr})

# Step 4: Save PCR summary
pcr_df = pd.DataFrame(pcr_results)
pcr_df.to_csv("reliance_pcr_summary.csv", index=False)
print("\nPut-Call Ratio Summary:")
print(pcr_df)

Saved reliance_option_chain_20Oct2025.csv with 91 rows.
Saved reliance_option_chain_28Oct2025.csv with 96 rows.
Saved reliance_option_chain_04Nov2025.csv with 91 rows.
Saved reliance_option_chain_11Nov2025.csv with 82 rows.
Saved reliance_option_chain_18Nov2025.csv with 80 rows.
Saved reliance_option_chain_25Nov2025.csv with 94 rows.
Saved reliance_option_chain_30Dec2025.csv with 106 rows.
Saved reliance_option_chain_31Mar2026.csv with 13 rows.
Saved reliance_option_chain_30Jun2026.csv with 18 rows.
Saved reliance_option_chain_29Sep2026.csv with 10 rows.
Saved reliance_option_chain_29Dec2026.csv with 20 rows.
Saved reliance_option_chain_29Jun2027.csv with 14 rows.
Saved reliance_option_chain_28Dec2027.csv with 16 rows.
Saved reliance_option_chain_27Jun2028.csv with 13 rows.
Saved reliance_option_chain_26Dec2028.csv with 14 rows.
Saved reliance_option_chain_26Jun2029.csv with 13 rows.
Saved reliance_option_chain_24Dec2029.csv with 13 rows.
Saved reliance_option_chain_25Jun2030.csv with 

In [7]:
import os
import time
import math
import json
import random
import string
import shutil
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

# ==========================
# CONFIG
# ==========================
BASE_DIR = "data"  # root folder for output
WARMUP_URL = "https://www.nseindia.com/option-chain"  # cookie warm-up page
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL  = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"

# Your big list (you can add/remove tickers here)
RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    'NSE:ATGL', 'NSE:ABCAPITAL', 'NSE:ABFRL', 'NSE:ALKEM', 'NSE:AMBER', 'NSE:AMBUJACEM',
    'NSE:ANGELONE', 'NSE:APLAPOLLO', 'NSE:APOLLOHOSP', 'NSE:ASHOKLEY', 'NSE:ASIANPAINT',
    'NSE:ASTRAL', 'NSE:AUBANK', 'NSE:AUROPHARMA', 'NSE:DMART', 'NSE:AXISBANK', 'NSE:BAJAJ-AUTO',
    'NSE:BAJFINANCE', 'NSE:BAJAJFINSV', 'NSE:BANDHANBNK', 'NSE:BANKBARODA', 'NSE:BANKINDIA',
    'NSE:BDL', 'NSE:BEL', 'NSE:BHARATFORG', 'NSE:BHEL', 'NSE:BPCL', 'NSE:BHARTIARTL',
    'NSE:BIOCON', 'NSE:BLUESTARCO', 'NSE:BOSCHLTD', 'NSE:BRITANNIA', 'NSE:BSE', 'NSE:CANBK',
    'NSE:CDSL', 'NSE:CESC', 'NSE:CGPOWER', 'NSE:CHOLAFIN', 'NSE:CIPLA', 'NSE:COALINDIA',
    'NSE:COFORGE', 'NSE:COLPAL', 'NSE:CAMS', 'NSE:CONCOR', 'NSE:CROMPTON', 'NSE:CUMMINSIND',
    'NSE:CYIENT', 'NSE:DABUR', 'NSE:DALBHARAT', 'NSE:DELHIVERY', 'NSE:DIVISLAB', 'NSE:DIXON',
    'NSE:DLF', 'NSE:DRREDDY', 'NSE:EICHERMOT', 'NSE:ETERNAL', 'NSE:EXIDEIND', 'NSE:FORTIS',
    'NSE:NYKAA', 'NSE:GAIL', 'NSE:GLENMARK', 'NSE:GMRAIRPORT', 'NSE:GODREJCP', 'NSE:GODREJPROP',
    'NSE:GRANULES', 'NSE:GRASIM', 'NSE:HAVELLS', 'NSE:HCLTECH', 'NSE:HDFCAMC', 'NSE:HDFCBANK',
    'NSE:HDFCLIFE', 'NSE:HEROMOTOCO', 'NSE:HFCL', 'NSE:HINDALCO', 'NSE:HAL', 'NSE:HINDPETRO',
    'NSE:HINDUNILVR', 'NSE:HINDZINC', 'NSE:HUDCO', 'NSE:ICICIBANK', 'NSE:ICICIGI', 'NSE:ICICIPRULI',
    'NSE:IDEA', 'NSE:IDFCFIRSTB', 'NSE:IIFL', 'NSE:INDIANB', 'NSE:IEX', 'NSE:IOC', 'NSE:IRCTC',
    'NSE:IRFC', 'NSE:IREDA', 'NSE:IGL', 'NSE:INDUSTOWER', 'NSE:INDUSINDBK', 'NSE:NAUKRI',
    'NSE:INFY', 'NSE:INOXWIND', 'NSE:INDIGO', 'NSE:IRB', 'NSE:ITC', 'NSE:JSL', 'NSE:JINDALSTEL',
    'NSE:JIOFIN', 'NSE:JSWENERGY', 'NSE:JSWSTEEL', 'NSE:JUBLFOOD', 'NSE:KALYANKJIL', 'NSE:KAYNES',
    'NSE:KEI', 'NSE:KFINTECH', 'NSE:KOTAKBANK', 'NSE:KPITTECH', 'NSE:LTF', 'NSE:LT', 'NSE:LAURUSLABS',
    'NSE:LICHSGFIN', 'NSE:LICI', 'NSE:LTIM', 'NSE:LUPIN', 'NSE:LODHA', 'NSE:MANAPPURAM',
    'NSE:MANKIND', 'NSE:MARICO', 'NSE:MARUTI', 'NSE:MFSL', 'NSE:MAXHEALTH', 'NSE:MAZDOCK',
    'NSE:MOTHERSON', 'NSE:MPHASIS', 'NSE:MCX', 'NSE:MUTHOOTFIN', 'NSE:NATIONALUM', 'NSE:NBCC',
    'NSE:NCC', 'NSE:NESTLEIND', 'NSE:NHPC','NSE:NMDC', 'NSE:NTPC', 'NSE:NUVAMA',
    'NSE:OBEROIRLTY', 'NSE:ONGC', 'NSE:OIL', 'NSE:PAYTM', 'NSE:OFSS', 'NSE:PAGEIND', 'NSE:PATANJALI',
    'NSE:POLICYBZR', 'NSE:PERSISTENT', 'NSE:PETRONET', 'NSE:PGEL', 'NSE:PIIND', 'NSE:PIDILITIND',
    'NSE:PPLPHARMA', 'NSE:PNBHOUSING', 'NSE:POLYCAB', 'NSE:POONAWALLA', 'NSE:PFC', 'NSE:POWERGRID',
    'NSE:PRESTIGE', 'NSE:PNB', 'NSE:RVNL', 'NSE:RBLBANK', 'NSE:RELIANCE', 'NSE:RECLTD', 'NSE:SBICARD',
    'NSE:SBILIFE', 'NSE:SHREECEM', 'NSE:SHRIRAMFIN', 'NSE:SIEMENS', 'NSE:SJVN', 'NSE:SOLARINDS',
    'NSE:SONACOMS', 'NSE:SRF', 'NSE:SBIN', 'NSE:SAIL', 'NSE:SUNPHARMA', 'NSE:SUPREMEIND', 'NSE:SUZLON',
    'NSE:SYNGENE', 'NSE:TATACHEM', 'NSE:TCS', 'NSE:TATACONSUM', 'NSE:TATAELXSI', 'NSE:TATAMOTORS',
    'NSE:TATAPOWER', 'NSE:TATASTEEL', 'NSE:TATATECH', 'NSE:TECHM', 'NSE:FEDERALBNK', 'NSE:INDHOTEL',
    'NSE:PHOENIXLTD', 'NSE:TITAGARH', 'NSE:TITAN', 'NSE:TORNTPHARM', 'NSE:TORNTPOWER', 'NSE:TRENT',
    'NSE:TIINDIA', 'NSE:TVSMOTOR', 'NSE:ULTRACEMCO', 'NSE:UNIONBANK', 'NSE:UNITDSPR', 'NSE:UNOMINDA',
    'NSE:UPL', 'NSE:VBL', 'NSE:VEDL', 'NSE:VOLTAS', 'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

# If you also want indices, add plain names like "NIFTY", "BANKNIFTY", etc.
INDEX_SYMBOLS = {"NIFTY", "BANKNIFTY", "FINNIFTY", "MIDCPNIFTY", "NIFTYIT"}

# ==========================
# HTTP Helpers
# ==========================
def new_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=3, sleep_min=0.8, sleep_max=1.6):
    for i in range(retries):
        try:
            resp = session.get(WARMUP_URL, timeout=10)
            if resp.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(random.uniform(sleep_min, sleep_max))
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=4, backoff=1.2):
    """
    Returns parsed JSON (dict) for the symbol. Raises on final failure.
    """
    url = (INDEX_OC_URL if is_index else EQUITY_OC_URL).format(symbol=symbol)
    last_exc = None
    for attempt in range(retries):
        try:
            resp = session.get(url, timeout=15)
            if resp.status_code == 200:
                # Ensure JSON; if blocked sometimes HTML is returned
                try:
                    return resp.json()
                except Exception:
                    # print(resp.text[:500])  # uncomment for debugging
                    raise RuntimeError("Received non-JSON (likely blocked).")
            elif resp.status_code in (429, 503, 403):
                # rate/blocked -> wait and retry
                time.sleep(backoff * (attempt + 1))
                continue
        except Exception as e:
            last_exc = e
            time.sleep(backoff * (attempt + 1))
    raise RuntimeError(f"Failed to fetch {symbol}. Last error: {last_exc}")

# ==========================
# Processing Helpers
# ==========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    """
    Build list of rows for a given expiry from NSE JSON.
    """
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    """
    PCR (OI) = sum(Put_OI) / sum(Call_OI). Returns (total_call_oi, total_put_oi, pcr or None)
    """
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi  = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# Main Pipeline
# ==========================
def get_target_expiries(session):
    """
    Decide the three target expiries once (using RELIANCE).
    This guarantees exactly three master PCR files overall.
    """
    base_symbol = "RELIANCE"
    data = fetch_option_chain(session, base_symbol, is_index=False)
    expiries = data.get("records", {}).get("expiryDates", [])
    if len(expiries) < 3:
        raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {expiries}")
    return expiries[:3]

def process_symbol(session, raw_symbol, target_expiries):
    """
    Fetches, writes per-expiry CSVs for the symbol, and returns PCR results for target expiries.
    """
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS

    # Fetch JSON with retry/backoff; handle warmup if needed
    try:
        data = fetch_option_chain(session, symbol, is_index=is_index)
    except Exception:
        # try re-warm and fetch again
        warm_cookies(session)
        data = fetch_option_chain(session, symbol, is_index=is_index)

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    out_dir = os.path.join(BASE_DIR, symbol)
    ensure_dir(out_dir)

    pcr_out = []  # list of dicts: {Expiry, Symbol, TotalCallOI, TotalPutOI, PCR}

    for expiry in target_expiries:
        if expiry not in found_expiries:
            # skip if this symbol doesn't list that target expiry
            continue

        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue

        # Save per-ticker per-expiry CSV
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        df.to_csv(os.path.join(out_dir, fname), index=False)

        # Compute PCR
        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })

    return pcr_out

def main():
    # Prepare base folder
    ensure_dir(BASE_DIR)

    # Create one session shared by threads (safe for requests) — each thread still sleeps/jitters
    session = new_session()

    # Warm cookies once upfront (extra safety)
    warm_cookies(session)

    # Fix target expiries based on RELIANCE so we produce exactly 3 master files
    target_expiries = get_target_expiries(session)
    print("Target expiries:", target_expiries)

    # Multithreaded processing of all symbols
    max_workers = 8  # tune if needed
    futures = []
    results = {exp: [] for exp in target_expiries}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries))

        for fut in as_completed(futures):
            sym_pcrs = fut.result()
            for row in sym_pcrs:
                # row: {"Symbol","Expiry","Total_Call_OI","Total_Put_OI","PCR"}
                exp = row["Expiry"]
                if exp in results:
                    results[exp].append(row)

    # Write 3 master PCR files
    for exp in target_expiries:
        rows = results.get(exp, [])
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"PCR_{exp}.csv".replace("/", "-")
        df.to_csv(os.path.join(BASE_DIR, master_name), index=False)
        print(f"Wrote master PCR: {master_name} ({len(df)} symbols)")

# Thread worker wrapper to isolate per-thread sessions & retries a bit more
def process_symbol_threadsafe(raw_symbol, target_expiries):
    # each thread gets its own session for robustness
    s = new_session()
    warm_cookies(s)
    # gentle random jitter to avoid hammering
    time.sleep(random.uniform(0.2, 0.7))
    try:
        return process_symbol(s, raw_symbol, target_expiries)
    except Exception as e:
        # log and continue
        print(f"[ERROR] {raw_symbol}: {e}")
        return []

if __name__ == "__main__":
    main()

Target expiries: ['28-Oct-2025', '25-Nov-2025', '30-Dec-2025']
Wrote master PCR: PCR_28-Oct-2025.csv (206 symbols)
Wrote master PCR: PCR_25-Nov-2025.csv (206 symbols)
Wrote master PCR: PCR_30-Dec-2025.csv (205 symbols)


In [9]:
#!/usr/bin/env python3
import os
import sys
import time
import math
import json
import random
import string
import shutil
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

# ==========================
# CONFIG
# ==========================
# Robust base folder resolution: prefer __file__ but fall back to argv[0] or cwd
def get_base_root():
    if "__file__" in globals():
        return os.path.dirname(os.path.abspath(__file__))
    # If run in notebook or interactive, sys.argv[0] might be set to something useful
    try:
        argv0 = sys.argv[0]
        if argv0:
            p = os.path.abspath(argv0)
            if os.path.exists(p):
                return os.path.dirname(p)
    except Exception:
        pass
    # final fallback
    return os.getcwd()

BASE_ROOT = get_base_root()
BASE_DIR = os.path.join(BASE_ROOT, "data")  # Absolute path
WARMUP_URL = "https://www.nseindia.com/option-chain"  # cookie warm-up page
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL  = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"

RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    'NSE:ATGL', 'NSE:ABCAPITAL', 'NSE:ABFRL', 'NSE:ALKEM', 'NSE:AMBER', 'NSE:AMBUJACEM',
    'NSE:ANGELONE', 'NSE:APLAPOLLO', 'NSE:APOLLOHOSP', 'NSE:ASHOKLEY', 'NSE:ASIANPAINT',
    'NSE:ASTRAL', 'NSE:AUBANK', 'NSE:AUROPHARMA', 'NSE:DMART', 'NSE:AXISBANK', 'NSE:BAJAJ-AUTO',
    'NSE:BAJFINANCE', 'NSE:BAJAJFINSV', 'NSE:BANDHANBNK', 'NSE:BANKBARODA', 'NSE:BANKINDIA',
    'NSE:BDL', 'NSE:BEL', 'NSE:BHARATFORG', 'NSE:BHEL', 'NSE:BPCL', 'NSE:BHARTIARTL',
    'NSE:BIOCON', 'NSE:BLUESTARCO', 'NSE:BOSCHLTD', 'NSE:BRITANNIA', 'NSE:BSE', 'NSE:CANBK',
    'NSE:CDSL', 'NSE:CESC', 'NSE:CGPOWER', 'NSE:CHOLAFIN', 'NSE:CIPLA', 'NSE:COALINDIA',
    'NSE:COFORGE', 'NSE:COLPAL', 'NSE:CAMS', 'NSE:CONCOR', 'NSE:CROMPTON', 'NSE:CUMMINSIND',
    'NSE:CYIENT', 'NSE:DABUR', 'NSE:DALBHARAT', 'NSE:DELHIVERY', 'NSE:DIVISLAB', 'NSE:DIXON',
    'NSE:DLF', 'NSE:DRREDDY', 'NSE:EICHERMOT', 'NSE:ETERNAL', 'NSE:EXIDEIND', 'NSE:FORTIS',
    'NSE:NYKAA', 'NSE:GAIL', 'NSE:GLENMARK', 'NSE:GMRAIRPORT', 'NSE:GODREJCP', 'NSE:GODREJPROP',
    'NSE:GRANULES', 'NSE:GRASIM', 'NSE:HAVELLS', 'NSE:HCLTECH', 'NSE:HDFCAMC', 'NSE:HDFCBANK',
    'NSE:HDFCLIFE', 'NSE:HEROMOTOCO', 'NSE:HFCL', 'NSE:HINDALCO', 'NSE:HAL', 'NSE:HINDPETRO',
    'NSE:HINDUNILVR', 'NSE:HINDZINC', 'NSE:HUDCO', 'NSE:ICICIBANK', 'NSE:ICICIGI', 'NSE:ICICIPRULI',
    'NSE:IDEA', 'NSE:IDFCFIRSTB', 'NSE:IIFL', 'NSE:INDIANB', 'NSE:IEX', 'NSE:IOC', 'NSE:IRCTC',
    'NSE:IRFC', 'NSE:IREDA', 'NSE:IGL', 'NSE:INDUSTOWER', 'NSE:INDUSINDBK', 'NSE:NAUKRI',
    'NSE:INFY', 'NSE:INOXWIND', 'NSE:INDIGO', 'NSE:IRB', 'NSE:ITC', 'NSE:JSL', 'NSE:JINDALSTEL',
    'NSE:JIOFIN', 'NSE:JSWENERGY', 'NSE:JSWSTEEL', 'NSE:JUBLFOOD', 'NSE:KALYANKJIL', 'NSE:KAYNES',
    'NSE:KEI', 'NSE:KFINTECH', 'NSE:KOTAKBANK', 'NSE:KPITTECH', 'NSE:LTF', 'NSE:LT', 'NSE:LAURUSLABS',
    'NSE:LICHSGFIN', 'NSE:LICI', 'NSE:LTIM', 'NSE:LUPIN', 'NSE:LODHA', 'NSE:MANAPPURAM',
    'NSE:MANKIND', 'NSE:MARICO', 'NSE:MARUTI', 'NSE:MFSL', 'NSE:MAXHEALTH', 'NSE:MAZDOCK',
    'NSE:MOTHERSON', 'NSE:MPHASIS', 'NSE:MCX', 'NSE:MUTHOOTFIN', 'NSE:NATIONALUM', 'NSE:NBCC',
    'NSE:NCC', 'NSE:NESTLEIND', 'NSE:NHPC','NSE:NMDC', 'NSE:NTPC', 'NSE:NUVAMA',
    'NSE:OBEROIRLTY', 'NSE:ONGC', 'NSE:OIL', 'NSE:PAYTM', 'NSE:OFSS', 'NSE:PAGEIND', 'NSE:PATANJALI',
    'NSE:POLICYBZR', 'NSE:PERSISTENT', 'NSE:PETRONET', 'NSE:PGEL', 'NSE:PIIND', 'NSE:PIDILITIND',
    'NSE:PPLPHARMA', 'NSE:PNBHOUSING', 'NSE:POLYCAB', 'NSE:POONAWALLA', 'NSE:PFC', 'NSE:POWERGRID',
    'NSE:PRESTIGE', 'NSE:PNB', 'NSE:RVNL', 'NSE:RBLBANK', 'NSE:RELIANCE', 'NSE:RECLTD', 'NSE:SBICARD',
    'NSE:SBILIFE', 'NSE:SHREECEM', 'NSE:SHRIRAMFIN', 'NSE:SIEMENS', 'NSE:SJVN', 'NSE:SOLARINDS',
    'NSE:SONACOMS', 'NSE:SRF', 'NSE:SBIN', 'NSE:SAIL', 'NSE:SUNPHARMA', 'NSE:SUPREMEIND', 'NSE:SUZLON',
    'NSE:SYNGENE', 'NSE:TATACHEM', 'NSE:TCS', 'NSE:TATACONSUM', 'NSE:TATAELXSI', 'NSE:TATAMOTORS',
    'NSE:TATAPOWER', 'NSE:TATASTEEL', 'NSE:TATATECH', 'NSE:TECHM', 'NSE:FEDERALBNK', 'NSE:INDHOTEL',
    'NSE:PHOENIXLTD', 'NSE:TITAGARH', 'NSE:TITAN', 'NSE:TORNTPHARM', 'NSE:TORNTPOWER', 'NSE:TRENT',
    'NSE:TIINDIA', 'NSE:TVSMOTOR', 'NSE:ULTRACEMCO', 'NSE:UNIONBANK', 'NSE:UNITDSPR', 'NSE:UNOMINDA',
    'NSE:UPL', 'NSE:VBL', 'NSE:VEDL', 'NSE:VOLTAS', 'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

INDEX_SYMBOLS = {"NIFTY", "BANKNIFTY", "FINNIFTY", "MIDCPNIFTY", "NIFTYIT"}

# ==========================
# HTTP Helpers
# ==========================
def new_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=3, sleep_min=0.8, sleep_max=1.6):
    for i in range(retries):
        try:
            resp = session.get(WARMUP_URL, timeout=10)
            if resp.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(random.uniform(sleep_min, sleep_max))
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=4, backoff=1.2):
    """
    Returns parsed JSON (dict) for the symbol. Raises on final failure.
    """
    url = (INDEX_OC_URL if is_index else EQUITY_OC_URL).format(symbol=symbol)
    last_exc = None
    for attempt in range(retries):
        try:
            resp = session.get(url, timeout=15)
            if resp.status_code == 200:
                try:
                    return resp.json()
                except Exception:
                    # sometimes HTML or anti-bot page is returned
                    last_exc = RuntimeError("Received non-JSON (likely blocked).")
                    # try a warmup then retry
                    warm_cookies(session)
                    time.sleep(backoff * (attempt + 1))
                    continue
            elif resp.status_code in (429, 503, 403):
                time.sleep(backoff * (attempt + 1))
                continue
            else:
                last_exc = RuntimeError(f"HTTP {resp.status_code}")
        except Exception as e:
            last_exc = e
            time.sleep(backoff * (attempt + 1))
    raise RuntimeError(f"Failed to fetch {symbol}. Last error: {last_exc}")

# ==========================
# Processing Helpers
# ==========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi  = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# Main Pipeline
# ==========================
def get_target_expiries(session):
    base_symbol = "RELIANCE"
    data = fetch_option_chain(session, base_symbol, is_index=False)
    expiries = data.get("records", {}).get("expiryDates", [])
    if len(expiries) < 3:
        raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {expiries}")
    return expiries[:3]

def process_symbol(session, raw_symbol, target_expiries):
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS

    # try fetch, and try warming once if initial fetch fails
    try:
        data = fetch_option_chain(session, symbol, is_index=is_index)
    except Exception:
        warm_cookies(session)
        data = fetch_option_chain(session, symbol, is_index=is_index)

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    out_dir = os.path.join(BASE_DIR, symbol)
    ensure_dir(out_dir)

    pcr_out = []
    for expiry in target_expiries:
        if expiry not in found_expiries:
            continue
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        out_path = os.path.join(out_dir, fname)
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            # log but continue
            print(f"[ERROR] writing {out_path}: {e}")
        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })
    return pcr_out

def process_symbol_threadsafe(raw_symbol, target_expiries):
    s = new_session()
    warm_cookies(s)
    # jitter so multiple threads don't synchronize requests
    time.sleep(random.uniform(0.2, 0.7))
    try:
        return process_symbol(s, raw_symbol, target_expiries)
    except Exception as e:
        print(f"[ERROR] {raw_symbol}: {e}")
        traceback.print_exc()
        return []

def main():
    ensure_dir(BASE_DIR)
    print(f"Output directory: {os.path.abspath(BASE_DIR)}")

    session = new_session()
    # give the server an initial warmup request
    if not warm_cookies(session):
        print("[WARN] warmup did not return 200 — script will still attempt to fetch data but may be blocked.")

    target_expiries = get_target_expiries(session)
    print("Target expiries:", target_expiries)

    max_workers = 8
    futures = []
    results = {exp: [] for exp in target_expiries}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries))

        # iterate safely so one future error doesn't kill the whole loop
        for fut in as_completed(futures):
            try:
                sym_pcrs = fut.result()
            except Exception as e:
                print("[ERROR] worker raised:", e)
                traceback.print_exc()
                continue
            for row in sym_pcrs:
                exp = row.get("Expiry")
                if exp in results:
                    results[exp].append(row)

    # Write 3 master PCR files
    for exp in target_expiries:
        rows = results.get(exp, [])
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"PCR_{exp}.csv".replace("/", "-")
        master_path = os.path.join(BASE_DIR, master_name)
        try:
            df.to_csv(master_path, index=False)
            print(f"Wrote master PCR: {master_name} ({len(df)} symbols)")
        except Exception as e:
            print(f"[ERROR] writing master file {master_path}: {e}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL] Unhandled exception in main():", e)
        traceback.print_exc()
        sys.exit(1)

Output directory: C:\Users\sarda\anaconda3\Lib\site-packages\data
Target expiries: ['28-Oct-2025', '25-Nov-2025', '30-Dec-2025']
Wrote master PCR: PCR_28-Oct-2025.csv (206 symbols)
Wrote master PCR: PCR_25-Nov-2025.csv (206 symbols)
Wrote master PCR: PCR_30-Dec-2025.csv (205 symbols)


In [11]:
#!/usr/bin/env python3
import os
import sys
import time
import random
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

# ==========================
# CONFIG (saving to Desktop\monthly_options_view)
# ==========================
# NOTE: using a raw string for the Windows path provided by the user
BASE_DIR = os.path.abspath(r"C:\Users\sarda\Desktop\monthly_options_view")
WARMUP_URL = "https://www.nseindia.com/option-chain"
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL  = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"

RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    'NSE:ATGL', 'NSE:ABCAPITAL', 'NSE:ABFRL', 'NSE:ALKEM', 'NSE:AMBER', 'NSE:AMBUJACEM',
    'NSE:ANGELONE', 'NSE:APLAPOLLO', 'NSE:APOLLOHOSP', 'NSE:ASHOKLEY', 'NSE:ASIANPAINT',
    'NSE:ASTRAL', 'NSE:AUBANK', 'NSE:AUROPHARMA', 'NSE:DMART', 'NSE:AXISBANK', 'NSE:BAJAJ-AUTO',
    'NSE:BAJFINANCE', 'NSE:BAJAJFINSV', 'NSE:BANDHANBNK', 'NSE:BANKBARODA', 'NSE:BANKINDIA',
    'NSE:BDL', 'NSE:BEL', 'NSE:BHARATFORG', 'NSE:BHEL', 'NSE:BPCL', 'NSE:BHARTIARTL',
    'NSE:BIOCON', 'NSE:BLUESTARCO', 'NSE:BOSCHLTD', 'NSE:BRITANNIA', 'NSE:BSE', 'NSE:CANBK',
    'NSE:CDSL', 'NSE:CESC', 'NSE:CGPOWER', 'NSE:CHOLAFIN', 'NSE:CIPLA', 'NSE:COALINDIA',
    'NSE:COFORGE', 'NSE:COLPAL', 'NSE:CAMS', 'NSE:CONCOR', 'NSE:CROMPTON', 'NSE:CUMMINSIND',
    'NSE:CYIENT', 'NSE:DABUR', 'NSE:DALBHARAT', 'NSE:DELHIVERY', 'NSE:DIVISLAB', 'NSE:DIXON',
    'NSE:DLF', 'NSE:DRREDDY', 'NSE:EICHERMOT', 'NSE:ETERNAL', 'NSE:EXIDEIND', 'NSE:FORTIS',
    'NSE:NYKAA', 'NSE:GAIL', 'NSE:GLENMARK', 'NSE:GMRAIRPORT', 'NSE:GODREJCP', 'NSE:GODREJPROP',
    'NSE:GRANULES', 'NSE:GRASIM', 'NSE:HAVELLS', 'NSE:HCLTECH', 'NSE:HDFCAMC', 'NSE:HDFCBANK',
    'NSE:HDFCLIFE', 'NSE:HEROMOTOCO', 'NSE:HFCL', 'NSE:HINDALCO', 'NSE:HAL', 'NSE:HINDPETRO',
    'NSE:HINDUNILVR', 'NSE:HINDZINC', 'NSE:HUDCO', 'NSE:ICICIBANK', 'NSE:ICICIGI', 'NSE:ICICIPRULI',
    'NSE:IDEA', 'NSE:IDFCFIRSTB', 'NSE:IIFL', 'NSE:INDIANB', 'NSE:IEX', 'NSE:IOC', 'NSE:IRCTC',
    'NSE:IRFC', 'NSE:IREDA', 'NSE:IGL', 'NSE:INDUSTOWER', 'NSE:INDUSINDBK', 'NSE:NAUKRI',
    'NSE:INFY', 'NSE:INOXWIND', 'NSE:INDIGO', 'NSE:IRB', 'NSE:ITC', 'NSE:JSL', 'NSE:JINDALSTEL',
    'NSE:JIOFIN', 'NSE:JSWENERGY', 'NSE:JSWSTEEL', 'NSE:JUBLFOOD', 'NSE:KALYANKJIL', 'NSE:KAYNES',
    'NSE:KEI', 'NSE:KFINTECH', 'NSE:KOTAKBANK', 'NSE:KPITTECH', 'NSE:LTF', 'NSE:LT', 'NSE:LAURUSLABS',
    'NSE:LICHSGFIN', 'NSE:LICI', 'NSE:LTIM', 'NSE:LUPIN', 'NSE:LODHA', 'NSE:MANAPPURAM',
    'NSE:MANKIND', 'NSE:MARICO', 'NSE:MARUTI', 'NSE:MFSL', 'NSE:MAXHEALTH', 'NSE:MAZDOCK',
    'NSE:MOTHERSON', 'NSE:MPHASIS', 'NSE:MCX', 'NSE:MUTHOOTFIN', 'NSE:NATIONALUM', 'NSE:NBCC',
    'NSE:NCC', 'NSE:NESTLEIND', 'NSE:NHPC','NSE:NMDC', 'NSE:NTPC', 'NSE:NUVAMA',
    'NSE:OBEROIRLTY', 'NSE:ONGC', 'NSE:OIL', 'NSE:PAYTM', 'NSE:OFSS', 'NSE:PAGEIND', 'NSE:PATANJALI',
    'NSE:POLICYBZR', 'NSE:PERSISTENT', 'NSE:PETRONET', 'NSE:PGEL', 'NSE:PIIND', 'NSE:PIDILITIND',
    'NSE:PPLPHARMA', 'NSE:PNBHOUSING', 'NSE:POLYCAB', 'NSE:POONAWALLA', 'NSE:PFC', 'NSE:POWERGRID',
    'NSE:PRESTIGE', 'NSE:PNB', 'NSE:RVNL', 'NSE:RBLBANK', 'NSE:RELIANCE', 'NSE:RECLTD', 'NSE:SBICARD',
    'NSE:SBILIFE', 'NSE:SHREECEM', 'NSE:SHRIRAMFIN', 'NSE:SIEMENS', 'NSE:SJVN', 'NSE:SOLARINDS',
    'NSE:SONACOMS', 'NSE:SRF', 'NSE:SBIN', 'NSE:SAIL', 'NSE:SUNPHARMA', 'NSE:SUPREMEIND', 'NSE:SUZLON',
    'NSE:SYNGENE', 'NSE:TATACHEM', 'NSE:TCS', 'NSE:TATACONSUM', 'NSE:TATAELXSI', 'NSE:TATAMOTORS',
    'NSE:TATAPOWER', 'NSE:TATASTEEL', 'NSE:TATATECH', 'NSE:TECHM', 'NSE:FEDERALBNK', 'NSE:INDHOTEL',
    'NSE:PHOENIXLTD', 'NSE:TITAGARH', 'NSE:TITAN', 'NSE:TORNTPHARM', 'NSE:TORNTPOWER', 'NSE:TRENT',
    'NSE:TIINDIA', 'NSE:TVSMOTOR', 'NSE:ULTRACEMCO', 'NSE:UNIONBANK', 'NSE:UNITDSPR', 'NSE:UNOMINDA',
    'NSE:UPL', 'NSE:VBL', 'NSE:VEDL', 'NSE:VOLTAS', 'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

INDEX_SYMBOLS = {"NIFTY", "BANKNIFTY", "FINNIFTY", "MIDCPNIFTY", "NIFTYIT"}

# ==========================
# HTTP Helpers
# ==========================
def new_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=3, sleep_min=0.8, sleep_max=1.6):
    for i in range(retries):
        try:
            resp = session.get(WARMUP_URL, timeout=10)
            if resp.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(random.uniform(sleep_min, sleep_max))
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=4, backoff=1.2):
    url = (INDEX_OC_URL if is_index else EQUITY_OC_URL).format(symbol=symbol)
    last_exc = None
    for attempt in range(retries):
        try:
            resp = session.get(url, timeout=15)
            if resp.status_code == 200:
                try:
                    return resp.json()
                except Exception:
                    last_exc = RuntimeError("Received non-JSON (likely blocked).")
                    warm_cookies(session)
                    time.sleep(backoff * (attempt + 1))
                    continue
            elif resp.status_code in (429, 503, 403):
                time.sleep(backoff * (attempt + 1))
                continue
            else:
                last_exc = RuntimeError(f"HTTP {resp.status_code}")
        except Exception as e:
            last_exc = e
            time.sleep(backoff * (attempt + 1))
    raise RuntimeError(f"Failed to fetch {symbol}. Last error: {last_exc}")

# ==========================
# Processing Helpers
# ==========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi  = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# Main Pipeline
# ==========================
def get_target_expiries(session):
    base_symbol = "RELIANCE"
    data = fetch_option_chain(session, base_symbol, is_index=False)
    expiries = data.get("records", {}).get("expiryDates", [])
    if len(expiries) < 3:
        raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {expiries}")
    return expiries[:3]

def process_symbol(session, raw_symbol, target_expiries):
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS

    try:
        data = fetch_option_chain(session, symbol, is_index=is_index)
    except Exception:
        warm_cookies(session)
        data = fetch_option_chain(session, symbol, is_index=is_index)

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    out_dir = os.path.join(BASE_DIR, symbol)
    ensure_dir(out_dir)

    pcr_out = []
    for expiry in target_expiries:
        if expiry not in found_expiries:
            continue
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        out_path = os.path.join(out_dir, fname)
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")
        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })
    return pcr_out

def process_symbol_threadsafe(raw_symbol, target_expiries):
    s = new_session()
    warm_cookies(s)
    time.sleep(random.uniform(0.2, 0.7))
    try:
        return process_symbol(s, raw_symbol, target_expiries)
    except Exception as e:
        print(f"[ERROR] {raw_symbol}: {e}")
        traceback.print_exc()
        return []

def main():
    ensure_dir(BASE_DIR)
    print(f"Output directory: {os.path.abspath(BASE_DIR)}")

    session = new_session()
    if not warm_cookies(session):
        print("[WARN] warmup did not return 200 — script will still attempt to fetch data but may be blocked.")

    target_expiries = get_target_expiries(session)
    print("Target expiries:", target_expiries)

    max_workers = 8
    futures = []
    results = {exp: [] for exp in target_expiries}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries))

        for fut in as_completed(futures):
            try:
                sym_pcrs = fut.result()
            except Exception as e:
                print("[ERROR] worker raised:", e)
                traceback.print_exc()
                continue
            for row in sym_pcrs:
                exp = row.get("Expiry")
                if exp in results:
                    results[exp].append(row)

    for exp in target_expiries:
        rows = results.get(exp, [])
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"PCR_{exp}.csv".replace("/", "-")
        master_path = os.path.join(BASE_DIR, master_name)
        try:
            df.to_csv(master_path, index=False)
            print(f"Wrote master PCR: {master_name} ({len(df)} symbols)")
        except Exception as e:
            print(f"[ERROR] writing master file {master_path}: {e}")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL] Unhandled exception in main():", e)
        traceback.print_exc()
        sys.exit(1)

Output directory: C:\Users\sarda\Desktop\monthly_options_view
Target expiries: ['28-Oct-2025', '25-Nov-2025', '30-Dec-2025']
Wrote master PCR: PCR_28-Oct-2025.csv (206 symbols)
Wrote master PCR: PCR_25-Nov-2025.csv (206 symbols)
Wrote master PCR: PCR_30-Dec-2025.csv (205 symbols)


In [13]:
#!/usr/bin/env python3
"""
Save equities option-chain CSVs and master PCRs (existing logic),
and additionally fetch & save option-chain CSVs for indices (NIFTY & BANKNIFTY)
for ALL expiries into BASE_DIR/index_options/<INDEX>/ with a final PCR_<INDEX>.csv.
"""

import os
import sys
import time
import random
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

# ==========================
# CONFIG
# ==========================
# Save to Desktop\monthly_options_view (as you asked previously)
BASE_DIR = os.path.abspath(r"C:\Users\sarda\Desktop\monthly_options_view")
WARMUP_URL = "https://www.nseindia.com/option-chain"
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL  = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"
# alternate index URL (some endpoints include an extra slash before the query)
INDEX_OC_URL_ALT = "https://www.nseindia.com/api/option-chain-indices/?symbol={symbol}"

# equities ticker list (unchanged)
RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    # ... (truncated for readability here; use your full list) ...
    'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

# Indexes we will process fully
INDEX_LIST = ["NIFTY", "BANKNIFTY"]

# If you want to keep the same set used earlier for detection
INDEX_SYMBOLS = set(INDEX_LIST)

# ==========================
# HTTP Helpers
# ==========================
def new_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=3, sleep_min=0.8, sleep_max=1.6):
    for i in range(retries):
        try:
            resp = session.get(WARMUP_URL, timeout=10)
            if resp.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(random.uniform(sleep_min, sleep_max))
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=4, backoff=1.2):
    """
    Fetch NSE option-chain JSON for the given symbol.
    If is_index=True and the primary index URL form fails, try the alternate form.
    Raises RuntimeError on final failure.
    """
    urls_to_try = []
    if is_index:
        urls_to_try = [INDEX_OC_URL.format(symbol=symbol), INDEX_OC_URL_ALT.format(symbol=symbol)]
    else:
        urls_to_try = [EQUITY_OC_URL.format(symbol=symbol)]

    last_exc = None
    for url in urls_to_try:
        for attempt in range(retries):
            try:
                resp = session.get(url, timeout=15)
                if resp.status_code == 200:
                    try:
                        return resp.json()
                    except Exception:
                        last_exc = RuntimeError("Received non-JSON (likely blocked).")
                        # warm and retry
                        warm_cookies(session)
                        time.sleep(backoff * (attempt + 1))
                        continue
                elif resp.status_code in (429, 503, 403):
                    # rate-limited/blocked, backoff and retry
                    time.sleep(backoff * (attempt + 1))
                    continue
                else:
                    last_exc = RuntimeError(f"HTTP {resp.status_code} for {url}")
            except Exception as e:
                last_exc = e
                time.sleep(backoff * (attempt + 1))
        # if we exhausted attempts for this URL, move to next URL (if any)
    raise RuntimeError(f"Failed to fetch {symbol}. Last error: {last_exc}")

# ==========================
# Processing Helpers
# ==========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi  = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# Equities processing (same as before)
# ==========================
def process_symbol(session, raw_symbol, target_expiries, out_base):
    """
    Writes per-expiry CSVs for the equity symbol under out_base/<SYMBOL>/
    Returns list of PCR dicts for those expiries.
    """
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS  # usually False for equities

    try:
        data = fetch_option_chain(session, symbol, is_index=is_index)
    except Exception:
        # one warm attempt and retry
        warm_cookies(session)
        data = fetch_option_chain(session, symbol, is_index=is_index)

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_out = []
    for expiry in target_expiries:
        if expiry not in found_expiries:
            continue
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        out_path = os.path.join(out_dir, fname)
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")
        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })
    return pcr_out

def process_symbol_threadsafe(raw_symbol, target_expiries, out_base):
    s = new_session()
    warm_cookies(s)
    time.sleep(random.uniform(0.2, 0.7))
    try:
        return process_symbol(s, raw_symbol, target_expiries, out_base)
    except Exception as e:
        print(f"[ERROR] {raw_symbol}: {e}")
        traceback.print_exc()
        return []

# ==========================
# Index processing
# ==========================
def process_index_all_expiries(session, symbol, out_base):
    """
    Fetch index option-chain JSON and for ALL expiries:
      - write per-expiry CSV under out_base/<symbol>/
      - compute PCR per expiry, collect into master PCR_<symbol>.csv
    Returns list of PCR rows (dicts).
    """
    symbol = symbol.upper()
    # fetch using index URL logic (fetch_option_chain will handle alt URL)
    data = fetch_option_chain(session, symbol, is_index=True)

    expiries = data.get("records", {}).get("expiryDates", [])
    if not expiries:
        raise RuntimeError(f"No expiries found for index {symbol}")

    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_rows = []
    for expiry in expiries:
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            # still create an empty CSV header for transparency (optional)
            df_empty = pd.DataFrame(columns=[
                "Strike","Expiry","Call_OI","Call_LTP","Put_OI","Put_LTP",
                "CE_BidQty","CE_BidPrice","CE_AskQty","CE_AskPrice",
                "PE_BidQty","PE_BidPrice","PE_AskQty","PE_AskPrice",
                "CE_IV","PE_IV","Underlying","UnderlyingValue"
            ])
            out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
            try:
                df_empty.to_csv(out_path, index=False)
            except Exception as e:
                print(f"[ERROR] writing empty csv {out_path}: {e}")
            # compute PCR from empty => zeros
            pcr_rows.append({
                "Expiry": expiry,
                "Total_Call_OI": 0,
                "Total_Put_OI": 0,
                "PCR": None
            })
            continue

        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")

        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_rows.append({
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })

    # write master PCR file for this index
    master_df = pd.DataFrame(pcr_rows, columns=["Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
    master_name = f"PCR_{symbol}.csv".replace("/", "-")
    master_path = os.path.join(out_dir, master_name)
    try:
        master_df.sort_values("Expiry", inplace=True)
        master_df.to_csv(master_path, index=False)
        print(f"Wrote index master PCR: {master_path} ({len(master_df)} expiries)")
    except Exception as e:
        print(f"[ERROR] writing index master file {master_path}: {e}")

    return pcr_rows

# ==========================
# Main
# ==========================
def main():
    ensure_dir(BASE_DIR)
    print(f"Output directory: {os.path.abspath(BASE_DIR)}")

    # directory for equities per previous behavior
    equities_out_base = BASE_DIR  # equities saved under BASE_DIR/<SYMBOL>/

    # directory for indices
    indices_base = os.path.join(BASE_DIR, "index_options")
    ensure_dir(indices_base)

    session = new_session()
    if not warm_cookies(session):
        print("[WARN] warmup did not return 200 — script will still attempt to fetch data but may be blocked.")

    # --- Equities: determine target expiries using RELIANCE (same as before) ---
    # NOTE: we preserve your original approach: decide exactly three expiries from RELIANCE
    try:
        base_symbol = "RELIANCE"
        reliance_data = fetch_option_chain(session, base_symbol, is_index=False)
        target_expiries = reliance_data.get("records", {}).get("expiryDates", [])
        if len(target_expiries) < 3:
            raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {target_expiries}")
        target_expiries = target_expiries[:3]
    except Exception as e:
        print("[FATAL] Cannot determine target expiries from RELIANCE:", e)
        traceback.print_exc()
        return

    print("Equities target expiries (master files):", target_expiries)

    # Multithread equities
    max_workers = 8
    futures = []
    results = {exp: [] for exp in target_expiries}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries, equities_out_base))
        for fut in as_completed(futures):
            try:
                sym_pcrs = fut.result()
            except Exception as e:
                print("[ERROR] worker raised:", e)
                traceback.print_exc()
                continue
            for row in sym_pcrs:
                exp = row.get("Expiry")
                if exp in results:
                    results[exp].append(row)

    # Write equities master PCR files (3 files)
    for exp in target_expiries:
        rows = results.get(exp, [])
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"PCR_{exp}.csv".replace("/", "-")
        master_path = os.path.join(BASE_DIR, master_name)
        try:
            df.to_csv(master_path, index=False)
            print(f"Wrote master PCR: {master_path} ({len(df)} symbols)")
        except Exception as e:
            print(f"[ERROR] writing master equities file {master_path}: {e}")

    # --- Indices: process all expiries for each index ---
    # We'll do indexes sequentially (only two) to reduce risk of being blocked
    for idx in INDEX_LIST:
        try:
            print(f"Processing index: {idx}")
            s = new_session()
            warm_cookies(s)
            process_index_all_expiries(s, idx, indices_base)
        except Exception as e:
            print(f"[ERROR] processing index {idx}: {e}")
            traceback.print_exc()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL] Unhandled exception in main():", e)
        traceback.print_exc()
        sys.exit(1)

Output directory: C:\Users\sarda\Desktop\monthly_options_view
Equities target expiries (master files): ['28-Oct-2025', '25-Nov-2025', '30-Dec-2025']
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options_view\PCR_28-Oct-2025.csv (9 symbols)
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options_view\PCR_25-Nov-2025.csv (9 symbols)
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options_view\PCR_30-Dec-2025.csv (9 symbols)
Processing index: NIFTY
Wrote index master PCR: C:\Users\sarda\Desktop\monthly_options_view\index_options\NIFTY\PCR_NIFTY.csv (18 expiries)
Processing index: BANKNIFTY
Wrote index master PCR: C:\Users\sarda\Desktop\monthly_options_view\index_options\BANKNIFTY\PCR_BANKNIFTY.csv (5 expiries)


In [15]:
#!/usr/bin/env python3
"""
Combined equities + indices option-chain fetcher.

- Equities: per-symbol per-expiry CSVs + 3 master PCR files (based on RELIANCE expiries)
- Indices: NIFTY & BANKNIFTY -> all expiries saved under index_options/<INDEX>/ and a PCR_<INDEX>.csv
"""

import os
import sys
import time
import random
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

# ==========================
# CONFIG (saving to Desktop\monthly_options_view)
# ==========================
BASE_DIR = os.path.abspath(r"C:\Users\sarda\Desktop\monthly_options")
WARMUP_URL = "https://www.nseindia.com/option-chain"
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL  = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"
# alternate index URL (some endpoints include an extra slash before the query)
INDEX_OC_URL_ALT = "https://www.nseindia.com/api/option-chain-indices/?symbol={symbol}"

# Full equities ticker list (use your full list)
RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    'NSE:ATGL', 'NSE:ABCAPITAL', 'NSE:ABFRL', 'NSE:ALKEM', 'NSE:AMBER', 'NSE:AMBUJACEM',
    'NSE:ANGELONE', 'NSE:APLAPOLLO', 'NSE:APOLLOHOSP', 'NSE:ASHOKLEY', 'NSE:ASIANPAINT',
    'NSE:ASTRAL', 'NSE:AUBANK', 'NSE:AUROPHARMA', 'NSE:DMART', 'NSE:AXISBANK', 'NSE:BAJAJ-AUTO',
    'NSE:BAJFINANCE', 'NSE:BAJAJFINSV', 'NSE:BANDHANBNK', 'NSE:BANKBARODA', 'NSE:BANKINDIA',
    'NSE:BDL', 'NSE:BEL', 'NSE:BHARATFORG', 'NSE:BHEL', 'NSE:BPCL', 'NSE:BHARTIARTL',
    'NSE:BIOCON', 'NSE:BLUESTARCO', 'NSE:BOSCHLTD', 'NSE:BRITANNIA', 'NSE:BSE', 'NSE:CANBK',
    'NSE:CDSL', 'NSE:CESC', 'NSE:CGPOWER', 'NSE:CHOLAFIN', 'NSE:CIPLA', 'NSE:COALINDIA',
    'NSE:COFORGE', 'NSE:COLPAL', 'NSE:CAMS', 'NSE:CONCOR', 'NSE:CROMPTON', 'NSE:CUMMINSIND',
    'NSE:CYIENT', 'NSE:DABUR', 'NSE:DALBHARAT', 'NSE:DELHIVERY', 'NSE:DIVISLAB', 'NSE:DIXON',
    'NSE:DLF', 'NSE:DRREDDY', 'NSE:EICHERMOT', 'NSE:ETERNAL', 'NSE:EXIDEIND', 'NSE:FORTIS',
    'NSE:NYKAA', 'NSE:GAIL', 'NSE:GLENMARK', 'NSE:GMRAIRPORT', 'NSE:GODREJCP', 'NSE:GODREJPROP',
    'NSE:GRANULES', 'NSE:GRASIM', 'NSE:HAVELLS', 'NSE:HCLTECH', 'NSE:HDFCAMC', 'NSE:HDFCBANK',
    'NSE:HDFCLIFE', 'NSE:HEROMOTOCO', 'NSE:HFCL', 'NSE:HINDALCO', 'NSE:HAL', 'NSE:HINDPETRO',
    'NSE:HINDUNILVR', 'NSE:HINDZINC', 'NSE:HUDCO', 'NSE:ICICIBANK', 'NSE:ICICIGI', 'NSE:ICICIPRULI',
    'NSE:IDEA', 'NSE:IDFCFIRSTB', 'NSE:IIFL', 'NSE:INDIANB', 'NSE:IEX', 'NSE:IOC', 'NSE:IRCTC',
    'NSE:IRFC', 'NSE:IREDA', 'NSE:IGL', 'NSE:INDUSTOWER', 'NSE:INDUSINDBK', 'NSE:NAUKRI',
    'NSE:INFY', 'NSE:INOXWIND', 'NSE:INDIGO', 'NSE:IRB', 'NSE:ITC', 'NSE:JSL', 'NSE:JINDALSTEL',
    'NSE:JIOFIN', 'NSE:JSWENERGY', 'NSE:JSWSTEEL', 'NSE:JUBLFOOD', 'NSE:KALYANKJIL', 'NSE:KAYNES',
    'NSE:KEI', 'NSE:KFINTECH', 'NSE:KOTAKBANK', 'NSE:KPITTECH', 'NSE:LTF', 'NSE:LT', 'NSE:LAURUSLABS',
    'NSE:LICHSGFIN', 'NSE:LICI', 'NSE:LTIM', 'NSE:LUPIN', 'NSE:LODHA', 'NSE:MANAPPURAM',
    'NSE:MANKIND', 'NSE:MARICO', 'NSE:MARUTI', 'NSE:MFSL', 'NSE:MAXHEALTH', 'NSE:MAZDOCK',
    'NSE:MOTHERSON', 'NSE:MPHASIS', 'NSE:MCX', 'NSE:MUTHOOTFIN', 'NSE:NATIONALUM', 'NSE:NBCC',
    'NSE:NCC', 'NSE:NESTLEIND', 'NSE:NHPC','NSE:NMDC', 'NSE:NTPC', 'NSE:NUVAMA',
    'NSE:OBEROIRLTY', 'NSE:ONGC', 'NSE:OIL', 'NSE:PAYTM', 'NSE:OFSS', 'NSE:PAGEIND', 'NSE:PATANJALI',
    'NSE:POLICYBZR', 'NSE:PERSISTENT', 'NSE:PETRONET', 'NSE:PGEL', 'NSE:PIIND', 'NSE:PIDILITIND',
    'NSE:PPLPHARMA', 'NSE:PNBHOUSING', 'NSE:POLYCAB', 'NSE:POONAWALLA', 'NSE:PFC', 'NSE:POWERGRID',
    'NSE:PRESTIGE', 'NSE:PNB', 'NSE:RVNL', 'NSE:RBLBANK', 'NSE:RELIANCE', 'NSE:RECLTD', 'NSE:SBICARD',
    'NSE:SBILIFE', 'NSE:SHREECEM', 'NSE:SHRIRAMFIN', 'NSE:SIEMENS', 'NSE:SJVN', 'NSE:SOLARINDS',
    'NSE:SONACOMS', 'NSE:SRF', 'NSE:SBIN', 'NSE:SAIL', 'NSE:SUNPHARMA', 'NSE:SUPREMEIND', 'NSE:SUZLON',
    'NSE:SYNGENE', 'NSE:TATACHEM', 'NSE:TCS', 'NSE:TATACONSUM', 'NSE:TATAELXSI', 'NSE:TATAMOTORS',
    'NSE:TATAPOWER', 'NSE:TATASTEEL', 'NSE:TATATECH', 'NSE:TECHM', 'NSE:FEDERALBNK', 'NSE:INDHOTEL',
    'NSE:PHOENIXLTD', 'NSE:TITAGARH', 'NSE:TITAN', 'NSE:TORNTPHARM', 'NSE:TORNTPOWER', 'NSE:TRENT',
    'NSE:TIINDIA', 'NSE:TVSMOTOR', 'NSE:ULTRACEMCO', 'NSE:UNIONBANK', 'NSE:UNITDSPR', 'NSE:UNOMINDA',
    'NSE:UPL', 'NSE:VBL', 'NSE:VEDL', 'NSE:VOLTAS', 'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

# Indexes we will fully process
INDEX_LIST = ["NIFTY", "BANKNIFTY"]
INDEX_SYMBOLS = set(INDEX_LIST)

# ==========================
# HTTP Helpers
# ==========================
def new_session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=3, sleep_min=0.8, sleep_max=1.6):
    for i in range(retries):
        try:
            resp = session.get(WARMUP_URL, timeout=10)
            if resp.status_code == 200:
                return True
        except Exception:
            pass
        time.sleep(random.uniform(sleep_min, sleep_max))
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=4, backoff=1.2):
    """
    Fetch NSE option-chain JSON for the given symbol.
    If is_index=True, try the two index URL variants if needed.
    Raises RuntimeError on final failure.
    """
    urls = []
    if is_index:
        urls = [INDEX_OC_URL.format(symbol=symbol), INDEX_OC_URL_ALT.format(symbol=symbol)]
    else:
        urls = [EQUITY_OC_URL.format(symbol=symbol)]

    last_exc = None
    for url in urls:
        for attempt in range(retries):
            try:
                resp = session.get(url, timeout=15)
                if resp.status_code == 200:
                    try:
                        return resp.json()
                    except Exception:
                        last_exc = RuntimeError("Received non-JSON (likely blocked).")
                        warm_cookies(session)
                        time.sleep(backoff * (attempt + 1))
                        continue
                elif resp.status_code in (429, 503, 403):
                    # rate-limited/blocked — backoff and retry
                    time.sleep(backoff * (attempt + 1))
                    continue
                else:
                    last_exc = RuntimeError(f"HTTP {resp.status_code} for {url}")
            except Exception as e:
                last_exc = e
                time.sleep(backoff * (attempt + 1))
        # exhausted attempts for this url -> try next url (if present)
    raise RuntimeError(f"Failed to fetch {symbol}. Last error: {last_exc}")

# ==========================
# Processing Helpers
# ==========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi  = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# Equities processing
# ==========================
def process_symbol(session, raw_symbol, target_expiries, out_base):
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS

    try:
        data = fetch_option_chain(session, symbol, is_index=is_index)
    except Exception:
        warm_cookies(session)
        data = fetch_option_chain(session, symbol, is_index=is_index)

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_out = []
    for expiry in target_expiries:
        if expiry not in found_expiries:
            continue
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        out_path = os.path.join(out_dir, fname)
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")
        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })
    return pcr_out

def process_symbol_threadsafe(raw_symbol, target_expiries, out_base):
    s = new_session()
    warm_cookies(s)
    time.sleep(random.uniform(0.2, 0.7))
    try:
        return process_symbol(s, raw_symbol, target_expiries, out_base)
    except Exception as e:
        print(f"[ERROR] {raw_symbol}: {e}")
        traceback.print_exc()
        return []

# ==========================
# Index processing
# ==========================
def process_index_all_expiries(session, symbol, out_base):
    symbol = symbol.upper()
    data = fetch_option_chain(session, symbol, is_index=True)

    expiries = data.get("records", {}).get("expiryDates", [])
    if not expiries:
        raise RuntimeError(f"No expiries found for index {symbol}")

    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_rows = []
    for expiry in expiries:
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            # create empty CSV header for transparency
            df_empty = pd.DataFrame(columns=[
                "Strike","Expiry","Call_OI","Call_LTP","Put_OI","Put_LTP",
                "CE_BidQty","CE_BidPrice","CE_AskQty","CE_AskPrice",
                "PE_BidQty","PE_BidPrice","PE_AskQty","PE_AskPrice",
                "CE_IV","PE_IV","Underlying","UnderlyingValue"
            ])
            out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
            try:
                df_empty.to_csv(out_path, index=False)
            except Exception as e:
                print(f"[ERROR] writing empty csv {out_path}: {e}")
            pcr_rows.append({
                "Expiry": expiry,
                "Total_Call_OI": 0,
                "Total_Put_OI": 0,
                "PCR": None
            })
            continue

        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            print(f"[ERROR] writing {out_path}: {e}")

        tcoi, tpoi, pcr = compute_pcr(rows)
        pcr_rows.append({
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr
        })

    master_df = pd.DataFrame(pcr_rows, columns=["Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
    master_name = f"PCR_{symbol}.csv".replace("/", "-")
    master_path = os.path.join(out_dir, master_name)
    try:
        master_df.sort_values("Expiry", inplace=True)
        master_df.to_csv(master_path, index=False)
        print(f"Wrote index master PCR: {master_path} ({len(master_df)} expiries)")
    except Exception as e:
        print(f"[ERROR] writing index master file {master_path}: {e}")

    return pcr_rows

# ==========================
# Main
# ==========================
def main():
    ensure_dir(BASE_DIR)
    print(f"Output directory: {os.path.abspath(BASE_DIR)}")

    equities_out_base = BASE_DIR
    indices_base = os.path.join(BASE_DIR, "index_options")
    ensure_dir(indices_base)

    session = new_session()
    if not warm_cookies(session):
        print("[WARN] warmup did not return 200 — script will still attempt to fetch data but may be blocked.")

    # --- Equities: determine three target expiries using RELIANCE ---
    try:
        base_symbol = "RELIANCE"
        reliance_data = fetch_option_chain(session, base_symbol, is_index=False)
        target_expiries = reliance_data.get("records", {}).get("expiryDates", [])
        if len(target_expiries) < 3:
            raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {target_expiries}")
        target_expiries = target_expiries[:3]
    except Exception as e:
        print("[FATAL] Cannot determine target expiries from RELIANCE:", e)
        traceback.print_exc()
        return

    print("Equities target expiries (master files):", target_expiries)

    # Multithread equities
    max_workers = 8
    futures = []
    results = {exp: [] for exp in target_expiries}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries, equities_out_base))
        for fut in as_completed(futures):
            try:
                sym_pcrs = fut.result()
            except Exception as e:
                print("[ERROR] worker raised:", e)
                traceback.print_exc()
                continue
            for row in sym_pcrs:
                exp = row.get("Expiry")
                if exp in results:
                    results[exp].append(row)

    # Write equities master PCR files (3 files)
    for exp in target_expiries:
        rows = results.get(exp, [])
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"PCR_{exp}.csv".replace("/", "-")
        master_path = os.path.join(BASE_DIR, master_name)
        try:
            df.to_csv(master_path, index=False)
            print(f"Wrote master PCR: {master_path} ({len(df)} symbols)")
        except Exception as e:
            print(f"[ERROR] writing master equities file {master_path}: {e}")

    # --- Indices: process all expiries for each index sequentially ---
    for idx in INDEX_LIST:
        try:
            print(f"Processing index: {idx}")
            s = new_session()
            warm_cookies(s)
            process_index_all_expiries(s, idx, indices_base)
        except Exception as e:
            print(f"[ERROR] processing index {idx}: {e}")
            traceback.print_exc()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL] Unhandled exception in main():", e)
        traceback.print_exc()
        sys.exit(1)

Output directory: C:\Users\sarda\Desktop\monthly_options
Equities target expiries (master files): ['28-Oct-2025', '25-Nov-2025', '30-Dec-2025']
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options\PCR_28-Oct-2025.csv (206 symbols)
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options\PCR_25-Nov-2025.csv (206 symbols)
Wrote master PCR: C:\Users\sarda\Desktop\monthly_options\PCR_30-Dec-2025.csv (205 symbols)
Processing index: NIFTY
Wrote index master PCR: C:\Users\sarda\Desktop\monthly_options\index_options\NIFTY\PCR_NIFTY.csv (18 expiries)
Processing index: BANKNIFTY
Wrote index master PCR: C:\Users\sarda\Desktop\monthly_options\index_options\BANKNIFTY\PCR_BANKNIFTY.csv (5 expiries)
